In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
with open('time_series.pkl', 'rb') as f:
    data = pkl.load(f)
    
with open('indices.pkl', 'rb') as f:
    indices = pkl.load(f)

In [ ]:
data_size = data.shape[0]
window_size = 20
print(data_size, data.shape, window_size)

## Prompt GPT 4

In [ ]:
API_KEY = ''

In [ ]:
system_prompt = f"Your job is to act as a professional finance analyst. You will write a high-quality report that is informative and helps in understanding the current financial situation."
print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for i in indices:
    
    data_window = data[i:i+window_size]
    
    s_p_500 = '|'.join([f"{x:.2f}" for x in data_window[:,1]])
    vix = '|'.join([f"{x:.2f}" for x in data_window[:,2]])
    nikkei_225 = '|'.join([f"{x:.2f}" for x in data_window[:,3]])
    ftse_100 = '|'.join([f"{x:.2f}" for x in data_window[:,4]])
    gold_futures = '|'.join([f"{x:.2f}" for x in data_window[:,5]])
    crude_oil_futures = '|'.join([f"{x:.2f}" for x in data_window[:,6]])
    eur_usd = '|'.join([f"{x:.2f}" for x in data_window[:,7]])
    usd_jpy = '|'.join([f"{x:.2f}" for x in data_window[:,8]])
    usd_cny = '|'.join([f"{x:.2f}" for x in data_window[:,9]])
    
    user_prompt = f"Your task is to analyze key financial indicators over the last {window_size} market days."
    user_prompt += f"\n\nReview the time-series data provided for the last {window_size} market days. "
    user_prompt += f"Each time-series consists of daily values separated by a \'|\' token for the following indicators:\n"
    user_prompt += f"- S&P 500: {s_p_500}\n- VIX (Volatility Index): {vix}\n- Nikkei 225: {nikkei_225}\n- FTSE 100: {ftse_100}\n- Gold Futures: {gold_futures}\n- Crude Oil Futures: {crude_oil_futures}\n- Exchange rate for EUR/USD: {eur_usd}\n- Exchange rate for USD/JYP: {usd_jpy}\n- Exchange rate for USD/CNY: {usd_cny}\n\n"
    user_prompt += f"Based on this time-series data, write a concise report that provides insights crucial for understanding the current financial situation. "
    user_prompt += f"Your report should be limited to five sentences, yet comprehensive, highlighting key trends and considering their potential impact on the market."
    user_prompt += f"Do not write numerical values while writing the report."
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content
    
    with open(f'gpt_summary/{i}.txt', 'w') as f:
        f.write(text)